In [6]:
import sys
import os
import pandas as pd
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))
from utils.utils import save_experiment, train_and_evaluate_logistic_regression, evaluate_model_on_parquet, grid_search_model
from configs.config_local import DATASET_PATH, ITW_DATASET_PATH, MODELS_PATH


from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SequentialFeatureSelector
import numpy as np


In [2]:
train_data_path = os.path.join(DATASET_PATH, "training_features_40_512_256_128.parquet")
test_data_path = os.path.join(DATASET_PATH, "testing_features_40_512_256_128.parquet")

In [ ]:


df = pd.read_parquet(train_data_path)
df_test = pd.read_parquet(test_data_path)
#print(df.head())
print(df_test.head())
print(df_test.shape)
#df.shape()

lr_params = {
            "C": 10.0,  # Regularization strength
            "class_weight": "balanced",  # Handle imbalanced classes
            "max_iter": 1000,  # Usually enough to converge
            "random_state": 42,
            "solver": "liblinear",  # Good for small-medium datasets, handles binary classification well
            "penalty": "l2",  # Standard L2 regularization
        }

pipeline, metrics, lr_params, feature_names, metadata_extra = train_and_evaluate_logistic_regression(train_data_path, test_data_path, lr_params)

  label                                           filename  mfcc_mean_0  \
0  real    file1.wav_16k.wav_norm.wav_mono.wav_silence.wav  -335.984039   
1  real   file10.wav_16k.wav_norm.wav_mono.wav_silence.wav  -335.599396   
2  real  file100.wav_16k.wav_norm.wav_mono.wav_silence.wav  -309.998932   
3  real  file1000.wav_16k.wav_norm.wav_mono.wav_silence...  -329.094482   
4  real  file1001.wav_16k.wav_norm.wav_mono.wav_silence...  -369.833069   

   mfcc_std_0  mfcc_mean_1  mfcc_std_1  mfcc_mean_2  mfcc_std_2  mfcc_mean_3  \
0  106.660751   134.430344   62.437195     9.798609   39.744644    13.036348   
1  115.704865   106.857346   50.405762    12.835211   42.213852    37.456020   
2   72.639381   143.500931   32.701843    16.544865   31.033148    49.439816   
3  119.318130    90.687096   70.433701    12.409759   38.385540    40.940025   
4  129.029114    77.314659   67.398788     2.168967   48.882267    34.795979   

   mfcc_std_3  ...  mel_spectrogram_mean_123  mel_spectrogram_std_12

In [5]:
print(metrics)

{'accuracy': 0.8109624514458351, 'precision': 0.8182309995432272, 'recall': 0.8126612097627887, 'f1': 0.810374659369997, 'roc_auc': 0.8960575360438938}


In [4]:
save_experiment(
    model=pipeline,
    metrics=metrics,
    experiment_dir=os.path.join(sys.path[0], "notebooks", "experiments",  "logistic_reg",),
    model_params=lr_params,
    feature_names=feature_names,
    metadata_extra=metadata_extra,
)


Experiment saved to: c:\Users\konst\Documents\GitHub\audio-deepfake-detection\notebooks\experiments\logistic_reg\exp_20260111_195634


'c:\\Users\\konst\\Documents\\GitHub\\audio-deepfake-detection\\notebooks\\experiments\\logistic_reg\\exp_20260111_195634'

In [ ]:
# test model on itw data
test_path = os.path.join(ITW_DATASET_PATH, 'normalized_features',"itw_features_40_2048_512_128_trimmed.parquet")
metrics,_=evaluate_model_on_parquet(pipeline, test_path)
metrics


{'accuracy': 0.6658306612908347,
 'precision': 0.5711204449994324,
 'recall': 0.4271160539943968,
 'f1': 0.6202675598200457,
 'roc_auc': 0.6838180253233701}

#### Train model without mel spectrogram featuress

In [6]:
train_data_path = os.path.join(DATASET_PATH, "training_features_40_512_256_128.parquet")
test_data_path = os.path.join(DATASET_PATH, "testing_features_40_512_256_128.parquet")

train_data = pd.read_parquet(train_data_path)
test_data = pd.read_parquet(test_data_path)
print(train_data.columns)
print('---------------------------------------')
print(test_data.columns)

Index(['label', 'filename', 'mfcc_mean_0', 'mfcc_std_0', 'mfcc_mean_1',
       'mfcc_std_1', 'mfcc_mean_2', 'mfcc_std_2', 'mfcc_mean_3', 'mfcc_std_3',
       ...
       'mel_spectrogram_mean_123', 'mel_spectrogram_std_123',
       'mel_spectrogram_mean_124', 'mel_spectrogram_std_124',
       'mel_spectrogram_mean_125', 'mel_spectrogram_std_125',
       'mel_spectrogram_mean_126', 'mel_spectrogram_std_126',
       'mel_spectrogram_mean_127', 'mel_spectrogram_std_127'],
      dtype='object', length=512)
---------------------------------------
Index(['label', 'filename', 'mfcc_mean_0', 'mfcc_std_0', 'mfcc_mean_1',
       'mfcc_std_1', 'mfcc_mean_2', 'mfcc_std_2', 'mfcc_mean_3', 'mfcc_std_3',
       ...
       'mel_spectrogram_mean_123', 'mel_spectrogram_std_123',
       'mel_spectrogram_mean_124', 'mel_spectrogram_std_124',
       'mel_spectrogram_mean_125', 'mel_spectrogram_std_125',
       'mel_spectrogram_mean_126', 'mel_spectrogram_std_126',
       'mel_spectrogram_mean_127', 'mel_spe

In [7]:
train_data_no_mel = train_data.loc[:, ~train_data.columns.str.startswith("mel_spectrogram")]
print(train_data_no_mel.columns)
print('---------------------------------------')
test_data_no_mel = test_data.loc[:, ~test_data.columns.str.startswith("mel_spectrogram")]
print(test_data_no_mel.columns)

Index(['label', 'filename', 'mfcc_mean_0', 'mfcc_std_0', 'mfcc_mean_1',
       'mfcc_std_1', 'mfcc_mean_2', 'mfcc_std_2', 'mfcc_mean_3', 'mfcc_std_3',
       ...
       'spectral_centroid_mean_0', 'spectral_centroid_std_0',
       'spectral_bandwidth_mean_0', 'spectral_bandwidth_std_0',
       'spectral_flatness_mean_0', 'spectral_flatness_std_0',
       'spectral_rolloff_mean_0', 'spectral_rolloff_std_0', 'pitch_yin_mean_0',
       'pitch_yin_std_0'],
      dtype='object', length=256)
---------------------------------------
Index(['label', 'filename', 'mfcc_mean_0', 'mfcc_std_0', 'mfcc_mean_1',
       'mfcc_std_1', 'mfcc_mean_2', 'mfcc_std_2', 'mfcc_mean_3', 'mfcc_std_3',
       ...
       'spectral_centroid_mean_0', 'spectral_centroid_std_0',
       'spectral_bandwidth_mean_0', 'spectral_bandwidth_std_0',
       'spectral_flatness_mean_0', 'spectral_flatness_std_0',
       'spectral_rolloff_mean_0', 'spectral_rolloff_std_0', 'pitch_yin_mean_0',
       'pitch_yin_std_0'],
      dtype=

In [8]:
save_train_no_mel_data_path = os.path.join(DATASET_PATH, "training_features_40_512_256_128_no_mel.parquet")
train_data_no_mel.to_parquet(save_train_no_mel_data_path)

save_test_no_mel_data_path = os.path.join(DATASET_PATH, "testing_features_40_512_256_128_no_mel.parquet")
test_data_no_mel.to_parquet(save_test_no_mel_data_path)


In [9]:
train_data_path = save_train_no_mel_data_path
test_data_path = save_test_no_mel_data_path

df = pd.read_parquet(train_data_path)
df_test = pd.read_parquet(test_data_path)
#print(df.head())
print(df_test.head())
print(df_test.shape)
#df.shape()

lr_params_no_mel = {
            "C": 1.0,  # Regularization strength
            "class_weight": "balanced",  # Handle imbalanced classes
            "max_iter": 1000,  # Usually enough to converge
            "random_state": 42,
            "solver": "liblinear",  # Good for small-medium datasets, handles binary classification well
            "penalty": "l2",  # Standard L2 regularization
        }

pipeline_no_mel, metrics_no_mel, lr_params_no_mel, feature_names_no_mel, metadata_extra_no_mel = train_and_evaluate_logistic_regression(train_data_path, test_data_path, lr_params)

  label                                           filename  mfcc_mean_0  \
0  real    file1.wav_16k.wav_norm.wav_mono.wav_silence.wav  -335.984039   
1  real   file10.wav_16k.wav_norm.wav_mono.wav_silence.wav  -335.599396   
2  real  file100.wav_16k.wav_norm.wav_mono.wav_silence.wav  -309.998932   
3  real  file1000.wav_16k.wav_norm.wav_mono.wav_silence...  -329.094482   
4  real  file1001.wav_16k.wav_norm.wav_mono.wav_silence...  -369.833069   

   mfcc_std_0  mfcc_mean_1  mfcc_std_1  mfcc_mean_2  mfcc_std_2  mfcc_mean_3  \
0  106.660751   134.430344   62.437195     9.798609   39.744644    13.036348   
1  115.704865   106.857346   50.405762    12.835211   42.213852    37.456020   
2   72.639381   143.500931   32.701843    16.544865   31.033148    49.439816   
3  119.318130    90.687096   70.433701    12.409759   38.385540    40.940025   
4  129.029114    77.314659   67.398788     2.168967   48.882267    34.795979   

   mfcc_std_3  ...  spectral_centroid_mean_0  spectral_centroid_std_

In [10]:
print(metrics_no_mel)

{'accuracy': 0.6793267155804921, 'precision': 0.7525714285714286, 'recall': 0.5556962025316455, 'f1': 0.6753322998468629, 'roc_auc': 0.7394593043193034}


In [11]:
save_experiment(
    model=pipeline_no_mel,
    metrics=metrics_no_mel,
    experiment_dir=os.path.join(sys.path[0], "notebooks", "experiments", "logistic_reg"),
    model_params=lr_params_no_mel,
    feature_names=feature_names_no_mel,
    metadata_extra=metadata_extra_no_mel,
)


Experiment saved to: c:\Users\konst\Documents\GitHub\audio-deepfake-detection\notebooks\experiments\logistic_reg\exp_20260111_195645


'c:\\Users\\konst\\Documents\\GitHub\\audio-deepfake-detection\\notebooks\\experiments\\logistic_reg\\exp_20260111_195645'

In [12]:
# test model on itw data

# remove mel features and save new df
test_data_path = os.path.join(ITW_DATASET_PATH, 'normalized_features',"itw_features_40_2048_512_128_trimmed.parquet")
test_data = pd.read_parquet(test_data_path)
test_data_no_mel = test_data.loc[:, ~test_data.columns.str.startswith("mel_spectrogram")]
save_test_data_no_mel_path = os.path.join(ITW_DATASET_PATH, 'normalized_features',"itw_features_40_2048_512_128_no_mel_trimmed.parquet")
test_data_no_mel.to_parquet(save_test_data_no_mel_path)

#evaluate the model
test_path = save_test_data_no_mel_path
metrics,_ = evaluate_model_on_parquet(pipeline_no_mel, test_path)
metrics

{'accuracy': 0.6264960792406108,
 'precision': 0.5005586592178771,
 'recall': 0.5324730452500213,
 'f1': 0.6059656349615231,
 'roc_auc': 0.6302691863007136}

### Fine tuning hyperparameters with cross validation

In [ ]:


model = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=5000)),
    ]
)

param_grid = {
    "clf__C": [0.01, 0.1, 1.0, 10.0],
    "clf__penalty": ["l2", "l1"],
    "clf__class_weight": [None, "balanced"],
    "clf__solver": ["lbfgs"], 
}

train_data_path = os.path.join(DATASET_PATH, "training_features_40_512_256_128.parquet")
test_data_path = os.path.join(DATASET_PATH, "testing_features_40_512_256_128.parquet")



best_model, metrics, best_params_, metadata, feature_names = grid_search_model(
    model,
    param_grid,
    train_data_path,
    test_data_path,
    scoring= "f1_macro",
    cv = 5,
    n_jobs= 1,
    verbose = 2,
)

print(best_params_)
print(metrics)


save_experiment(
    model=best_model,
    metrics=metrics,
    experiment_dir=os.path.join(sys.path[0], "notebooks", "experiments",  "logistic_reg",),
    model_params=best_params_,
    feature_names=feature_names,
    metadata_extra=metadata,
)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END clf__C=0.01, clf__class_weight=None, clf__penalty=l2, clf__solver=lbfgs; total time=   1.8s
[CV] END clf__C=0.01, clf__class_weight=None, clf__penalty=l2, clf__solver=lbfgs; total time=   1.8s
[CV] END clf__C=0.01, clf__class_weight=None, clf__penalty=l2, clf__solver=lbfgs; total time=   1.9s
[CV] END clf__C=0.01, clf__class_weight=None, clf__penalty=l2, clf__solver=lbfgs; total time=   2.2s
[CV] END clf__C=0.01, clf__class_weight=None, clf__penalty=l2, clf__solver=lbfgs; total time=   1.5s
[CV] END clf__C=0.01, clf__class_weight=None, clf__penalty=l1, clf__solver=lbfgs; total time=   0.4s
[CV] END clf__C=0.01, clf__class_weight=None, clf__penalty=l1, clf__solver=lbfgs; total time=   0.4s
[CV] END clf__C=0.01, clf__class_weight=None, clf__penalty=l1, clf__solver=lbfgs; total time=   0.4s
[CV] END clf__C=0.01, clf__class_weight=None, clf__penalty=l1, clf__solver=lbfgs; total time=   0.4s
[CV] END clf__C=0.01, clf__cla

c:\Users\konst\anaconda3\envs\audio_deepfake_py313\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
40 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\konst\anaconda3\envs\audio_deepfake_py313\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\konst\anaconda3\envs\audio_deepfake_py313\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\konst\anaconda3\envs\aud

{'clf__C': 1.0, 'clf__class_weight': 'balanced', 'clf__penalty': 'l2', 'clf__solver': 'lbfgs'}
{'accuracy': 0.8258523953388002, 'precision': 0.8303092480899548, 'recall': 0.8271687838260947, 'f1': 0.8255921637596269, 'roc_auc': 0.9051428709874609}
Experiment saved to: c:\Users\konst\Documents\GitHub\audio-deepfake-detection\notebooks\experiments\logistic_reg\exp_20260126_164821


'c:\\Users\\konst\\Documents\\GitHub\\audio-deepfake-detection\\notebooks\\experiments\\logistic_reg\\exp_20260126_164821'

Evaluate Best Model to ITW

In [7]:
itw_test_data_path = os.path.join(ITW_DATASET_PATH, 'normalized_features',"itw_features_40_2048_512_128_trimmed.parquet")
metrics, metadata_extra = evaluate_model_on_parquet(best_model, itw_test_data_path)
print(metrics)

{'accuracy': 0.6661798787263088, 'precision': 0.5765503875968992, 'recall': 0.4041090075558197, 'f1': 0.6152114882678517, 'roc_auc': 0.6846598215065247}


### Select features using logistic regression and forward/backward elimination

In [ ]:

train_data_path = os.path.join(DATASET_PATH, "training_features_40_512_256_128.parquet")
test_data_path = os.path.join(DATASET_PATH, "testing_features_40_512_256_128.parquet")

df_train = pd.read_parquet(train_data_path)
df_test = pd.read_parquet(test_data_path)

def split_xy(df):
    X = df.drop(columns=["label", "filename"], errors="ignore")
    y = df["label"].map({"real": 0, "fake": 1}).values
    if np.isnan(y).any():
        raise ValueError("Invalid label values detected")
    return X.values, y, X.columns.tolist()

for df in (df_train, df_test):
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)


X_train, y_train, _ = split_xy(df_train)
X_test, y_test, _ = split_xy(df_test)

model = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(C=1.0, class_weight= 'balanced', penalty="l2", solver='lbfgs',max_iter=5000)),
    ]
)

sfs = SequentialFeatureSelector(
    estimator=model,
    n_features_to_select='auto',
    direction="backward",
    scoring="f1_macro",
    tol=0.01,
    cv=3,
    n_jobs=1,
)


sfs.fit(X_train, y_train)



c:\Users\konst\anaconda3\envs\audio_deepfake_py313\Lib\site-packages\sklearn\model_selection\_validation.py:971: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\konst\anaconda3\envs\audio_deepfake_py313\Lib\site-packages\sklearn\metrics\_scorer.py", line 152, in __call__
    score = scorer._score(
        cached_call, estimator, *args, **routed_params.get(name).score
    )
  File "c:\Users\konst\anaconda3\envs\audio_deepfake_py313\Lib\site-packages\sklearn\metrics\_scorer.py", line 400, in _score
    y_pred = method_caller(
        estimator,
    ...<2 lines>...
        pos_label=pos_label,
    )
  File "c:\Users\konst\anaconda3\envs\audio_deepfake_py313\Lib\site-packages\sklearn\metrics\_scorer.py", line 90, in _cached_call
    result, _ = _get_response_values(
                ~~~~~~~~~~~~~~~~~~~~^
        estimator, *args, response_method=response_method, **kwarg

ValueError: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\konst\anaconda3\envs\audio_deepfake_py313\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\konst\anaconda3\envs\audio_deepfake_py313\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\konst\anaconda3\envs\audio_deepfake_py313\Lib\site-packages\sklearn\pipeline.py", line 655, in fit
    Xt = self._fit(X, y, routed_params, raw_params=params)
  File "c:\Users\konst\anaconda3\envs\audio_deepfake_py313\Li

In [20]:
train_data_path = os.path.join(DATASET_PATH, "training_features_40_512_256_128.parquet")
test_data_path = os.path.join(DATASET_PATH, "testing_features_40_512_256_128.parquet")

df_train = pd.read_parquet(train_data_path)
df_test = pd.read_parquet(test_data_path)
print(df_train.columns)

Index(['label', 'filename', 'mfcc_mean_0', 'mfcc_std_0', 'mfcc_mean_1',
       'mfcc_std_1', 'mfcc_mean_2', 'mfcc_std_2', 'mfcc_mean_3', 'mfcc_std_3',
       ...
       'mel_spectrogram_mean_123', 'mel_spectrogram_std_123',
       'mel_spectrogram_mean_124', 'mel_spectrogram_std_124',
       'mel_spectrogram_mean_125', 'mel_spectrogram_std_125',
       'mel_spectrogram_mean_126', 'mel_spectrogram_std_126',
       'mel_spectrogram_mean_127', 'mel_spectrogram_std_127'],
      dtype='object', length=512)


### Train model without standar deviation of features

In [28]:

train_data_path = os.path.join(DATASET_PATH, "training_features_40_512_256_128.parquet")
test_data_path = os.path.join(DATASET_PATH, "testing_features_40_512_256_128.parquet")

df_train = pd.read_parquet(train_data_path)
df_test = pd.read_parquet(test_data_path)

train_data_no_std = df_train.loc[:, ~df_train.columns.str.contains("_std_")]
print(train_data_no_std.columns)
print('---------------------------------------')
test_data_no_std = df_test.loc[:, ~df_test.columns.str.contains("_std_")]
print(test_data_no_std.columns)

train_data_no_std.to_parquet(os.path.join(DATASET_PATH, "training_features_no_std_40_512_256_128.parquet"))
test_data_no_std.to_parquet(os.path.join(DATASET_PATH, "testing_features_no_std_40_512_256_128.parquet"))



df = pd.read_parquet(os.path.join(DATASET_PATH, "training_features_no_std_40_512_256_128.parquet"))
df_test = pd.read_parquet(os.path.join(DATASET_PATH, "testing_features_no_std_40_512_256_128.parquet"))
#print(df.head())
print(df_test.head())
print(df_test.shape)
#df.shape()

lr_params = {
            "C": 1.0,  # Regularization strength
            "class_weight": "balanced",  # Handle imbalanced classes
            "max_iter": 5000,  # Usually enough to converge
            "random_state": 42,
            "solver": "lbfgs",  # Good for small-medium datasets, handles binary classification well
            "penalty": "l2",  # Standard L2 regularization
        }

pipeline_no_std, metrics_no_std, lr_params_no_std, feature_names_no_std, metadata_extra_no_std = train_and_evaluate_logistic_regression(os.path.join(DATASET_PATH, "training_features_no_std_40_512_256_128.parquet"),
                                                                                                                                         os.path.join(DATASET_PATH, "testing_features_no_std_40_512_256_128.parquet"),
                                                                                                                                         lr_params)

print(metrics_no_std)

Index(['label', 'filename', 'mfcc_mean_0', 'mfcc_mean_1', 'mfcc_mean_2',
       'mfcc_mean_3', 'mfcc_mean_4', 'mfcc_mean_5', 'mfcc_mean_6',
       'mfcc_mean_7',
       ...
       'mel_spectrogram_mean_118', 'mel_spectrogram_mean_119',
       'mel_spectrogram_mean_120', 'mel_spectrogram_mean_121',
       'mel_spectrogram_mean_122', 'mel_spectrogram_mean_123',
       'mel_spectrogram_mean_124', 'mel_spectrogram_mean_125',
       'mel_spectrogram_mean_126', 'mel_spectrogram_mean_127'],
      dtype='object', length=257)
---------------------------------------
Index(['label', 'filename', 'mfcc_mean_0', 'mfcc_mean_1', 'mfcc_mean_2',
       'mfcc_mean_3', 'mfcc_mean_4', 'mfcc_mean_5', 'mfcc_mean_6',
       'mfcc_mean_7',
       ...
       'mel_spectrogram_mean_118', 'mel_spectrogram_mean_119',
       'mel_spectrogram_mean_120', 'mel_spectrogram_mean_121',
       'mel_spectrogram_mean_122', 'mel_spectrogram_mean_123',
       'mel_spectrogram_mean_124', 'mel_spectrogram_mean_125',
       'mel_

In [31]:
itw_test_data_path = os.path.join(ITW_DATASET_PATH, 'normalized_features',"itw_features_40_2048_512_128_trimmed.parquet")
itw_data= pd.read_parquet(itw_test_data_path)
itw_data_no_std = itw_data.loc[:, ~itw_data.columns.str.contains("_std_")]
print(itw_data_no_std.columns)
itw_data_no_std.to_parquet(os.path.join(DATASET_PATH, "itw_features_no_std_40_512_256_128.parquet"))

metrics, metadata_extra = evaluate_model_on_parquet(pipeline_no_std, os.path.join(DATASET_PATH, "itw_features_no_std_40_512_256_128.parquet"))
print(metrics)

Index(['label', 'filename', 'mfcc_mean_0', 'mfcc_mean_1', 'mfcc_mean_2',
       'mfcc_mean_3', 'mfcc_mean_4', 'mfcc_mean_5', 'mfcc_mean_6',
       'mfcc_mean_7',
       ...
       'mel_spectrogram_mean_118', 'mel_spectrogram_mean_119',
       'mel_spectrogram_mean_120', 'mel_spectrogram_mean_121',
       'mel_spectrogram_mean_122', 'mel_spectrogram_mean_123',
       'mel_spectrogram_mean_124', 'mel_spectrogram_mean_125',
       'mel_spectrogram_mean_126', 'mel_spectrogram_mean_127'],
      dtype='object', length=257)
{'accuracy': 0.6257976443696626, 'precision': 0.4982456140350877, 'recall': 0.09644282197130487, 'f1': 0.46037690545200727, 'roc_auc': 0.6134876900428049}
